Para este caso retomamos los datos guardados en la tabla "apartamentRental"

# Consultamos los datos

Importamos librerias y configuramos la conexión con postgres

In [1]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

#path del archivo postgresql-42.1.4.jar
dir = "/Users/jasonsolano/Documents/BigData/BigDataTEC/Clase1/DB/"
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .getOrCreate()

Consultamos los datos

In [2]:
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ImmoDataBase") \
    .option("user", "postgres") \
    .option("password", "102800") \
    .option("dbtable", "apartmentrental") \
    .load()

df.limit(5).toPandas().head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,electricityBasePrice,picturecount,pricetrend,...,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,heatingCosts,energyEfficiencyClass,lastRefurbish,date
0,Baden_Württemberg,350.0,floor_heating,ONE_YEAR_FREE,NaN,True,True,NaN,12,5.07,...,NaN,3,False,6,Reutlingen_Kreis,Pfullingen,None,A,None,May19
1,Nordrhein_Westfalen,100.0,None,ONE_YEAR_FREE,10.0,False,False,NaN,29,3.52,...,NaN,2,False,2,Essen,Karnap,None,None,None,May19
2,Sachsen,90.0,None,ONE_YEAR_FREE,10.0,False,False,NaN,11,0.59,...,NaN,2,False,2,Zwickau_Kreis,Werdau,None,None,None,May19
3,Nordrhein_Westfalen,84.0,central_heating,ONE_YEAR_FREE,10.0,False,True,90.76,4,4.92,...,2.0,3,False,3,Steinfurt_Kreis,Emsdetten,None,None,None,Sep18
4,Nordrhein_Westfalen,195.0,central_heating,ONE_YEAR_FREE,NaN,False,True,NaN,3,3.58,...,NaN,2,False,3,Bielefeld,Schildesche,None,None,None,Oct19


Observamos datos importantes

In [3]:
df.describe().toPandas().head()

,summary,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,electricityBasePrice,picturecount,pricetrend,telekomUploadSpeed,...,floor,numberOfFloors,noRoomsRange,livingSpaceRange,regio2,regio3,heatingCosts,energyEfficiencyClass,lastRefurbish,date
0,count,198379,193267,165769,174578,36865,47185,198379,197158,174011,...,160750,126549,198361,198361,198361,198361,63195,55008,59275,198361
1,mean,None,151.43279711487244,None,None,10.0,89.10167977102157,9.718145569843582,3.435082573367232,27.785562981649313,...,2.1272970451010886,3.571162158531478,2.580330810996113,3.0788713507191434,None,None,78.0102742305564,None,2013.5937579080557,None
2,stddev,None,352.8943402904954,None,None,0.0,5.413494724155707,6.348194348133843,2.055461804742117,16.655154583281252,...,4.014081150928973,6.349898372180542,0.9341394698673625,1.4058025434166923,None,None,159.5179245725537,None,10.11258258432115,None
3,min,Baden_Württemberg,0.0,central_heating,NONE,10.0,71.43,0,-9.17,1.0,...,-1.0,0.0,1,1,Aachen,Aach,0.0,A,1015.0,May19
4,max,Thüringen,146118.0,wood_pellet_heating,ON_DEMAND,10.0,90.76,109,14.92,100.0,...,999.0,999.0,5,7,Zwickau_Kreis,Ürzig,12613.0,NO_INFORMATION,2919.0,Sep18


In [5]:
df.printSchema()

root
 |-- regio1: string (nullable = true)
 |-- serviceCharge: double (nullable = true)
 |-- heatingType: string (nullable = true)
 |-- telekomTvOffer: string (nullable = true)
 |-- telekomHybridUploadSpeed: double (nullable = true)
 |-- newlyConst: boolean (nullable = true)
 |-- balcony: boolean (nullable = true)
 |-- electricityBasePrice: double (nullable = true)
 |-- picturecount: integer (nullable = true)
 |-- pricetrend: double (nullable = true)
 |-- telekomUploadSpeed: double (nullable = true)
 |-- totalRent: double (nullable = true)
 |-- yearConstructed: double (nullable = true)
 |-- electricityKwhPrice: double (nullable = true)
 |-- scoutId: integer (nullable = true)
 |-- noParkSpaces: double (nullable = true)
 |-- firingTypes: string (nullable = true)
 |-- hasKitchen: boolean (nullable = true)
 |-- geo_bln: string (nullable = true)
 |-- cellar: boolean (nullable = true)
 |-- yearConstructedRange: double (nullable = true)
 |-- baseRent: double (nullable = true)
 |-- houseNumber

Detectamos valores nulos para cada columna